In [1]:
# Imports

import open3d as o3d
import numpy as np
import cv2
import numpy as np
from ultralytics import YOLO
import copy

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# Data Path

model_data = r"C:\Hackathon\3D bone mapping\forearm_Bones.glb"
image_data = r"C:\Hackathon\3D bone mapping\working_code\test\test_5.png"

In [3]:
# Model and Image

mesh = o3d.io.read_triangle_mesh(model_data)
img = cv2.imread(image_data)

In [4]:
# Per-processing Model

mesh.remove_duplicated_vertices()
mesh.remove_degenerate_triangles()
mesh.remove_duplicated_triangles()
mesh.remove_non_manifold_edges()
mesh.remove_unreferenced_vertices()

mesh.scale(1 / np.max(mesh.get_max_bound() - mesh.get_min_bound()), center=mesh.get_center())
mesh.translate(-mesh.get_center())
mesh.compute_vertex_normals()

R1 = mesh.get_rotation_matrix_from_axis_angle([np.pi, 0, 0])
R2 = mesh.get_rotation_matrix_from_axis_angle([0, -np.pi / 2, 0])

R = R2 @ R1
mesh.rotate(R2, center=(0, 0, 0))

TriangleMesh with 485454 points and 970900 triangles.

In [5]:
# Bounding Box around model 

bbox = mesh.get_axis_aligned_bounding_box()
bbox.color = (1, 0, 0)

min_bound = bbox.get_min_bound()
max_bound = bbox.get_max_bound()
size = max_bound - min_bound

print(f"Width:  {size[0]:.4f}")
print(f"Height: {size[1]:.4f}")
print(f"Depth:  {size[2]:.4f}")
print("Model center:", mesh.get_center())

o3d.visualization.draw_geometries([mesh, bbox])

Width:  0.2318
Height: 1.0000
Depth:  0.1647
Model center: [ 3.4601e-14  1.0308e-12  6.0746e-13]


In [6]:
# Divide Mesh to make Clusters 

plane_height, plane_depth = size[1], size[2]
plane_thickness = 0.001

vertical_plane = o3d.geometry.TriangleMesh.create_box(
    width=plane_thickness,
    height=plane_height,
    depth=plane_depth
)

vertical_plane.translate((-plane_thickness/2, -plane_height/2, -plane_depth/2))

center_x = (min_bound[0] + max_bound[0]) / 2
vertical_plane.translate((center_x, 0, 0))

angle_from_x = 94.11222884471846
tilt_angle = angle_from_x - 90
angle_rad = np.deg2rad(-tilt_angle)

R = vertical_plane.get_rotation_matrix_from_axis_angle([0, 0, angle_rad])
vertical_plane.rotate(R, center=vertical_plane.get_center())

shift_amount = -0.015
vertical_plane.translate((shift_amount, 0, 0))

vertical_plane.paint_uniform_color([1, 0.7, 0.3])

o3d.visualization.draw_geometries([mesh, vertical_plane])

In [7]:
# Create Clusters

plane_normal = R @ np.array([1.0, 0.0, 0.0])
plane_normal /= np.linalg.norm(plane_normal)

plane_center = vertical_plane.get_center()
points = np.asarray(mesh.vertices)

signed_distances = np.dot(points - plane_center, plane_normal)

mask_above = signed_distances > 0
mask_below = signed_distances <= 0

mesh_ulna = mesh.select_by_index(np.where(mask_above)[0].tolist())
mesh_radius = mesh.select_by_index(np.where(mask_below)[0].tolist())

mesh_ulna.paint_uniform_color([0.2, 0.8, 1.0])
mesh_radius.paint_uniform_color([1.0, 0.4, 0.4])

o3d.visualization.draw_geometries([mesh_radius, mesh_ulna])